# Langchain 알아보기

### Part 1. Langchain - Prompt Template
- Objectives: Langchain 라이브러리를 활용하여 GPT 모델에 대한 프롬프트 엔지니어링 진행


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.environ.get('OPENAI_API_KEY')

# 프롬프트 엔지니어링이란?
### LLM 프롬프트 수정을 통해 일반 바닐라한 기능성에 대해 추가적인 기능을 부여
- 1) 인스트럭션
> LLM에게 무엇을 하기를 원하는지 요건을 적는 필드
- 2) 컨텍스트
> 답변 생성 시 참고하여 답을 하기를 원하는 정보를 적는 필드
- 3) 사용자 쿼리
> 사용자의 질문을 넣는 필드(템플릿 전체가 질문일 순 없으니)
- 4) 답변 필드
> 질문에 따른 답변을 하는 필드

In [5]:

prompt = """Please answer the question based on the context below. If you can't find the information to answer the question
from the context provided, please say "I don't know".

context: Bitcoin is the first decentralized cryptocurrency.
Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger,
called a blockchain, without central oversight.
Consensus between nodes is achieved using a computationally intensive process based on proof of work,
called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.
Based on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.
Use of bitcoin as a currency began in 2009, ith the release of its open-source implementation.
Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account.
It is mostly seen as an investment and has been described by many scholars as an economic bubble.
As bitcoin is pseudonymous, its use by criminals has attracted the attention of regulators,
leading to its ban by several countries as of 2021.

Question: how is bitcoin used?

Answer:"""

In [4]:
#랭체인에서 일반 채팅 모델(gpt completion)활용한 간단한 질의 구성 래퍼
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    api_key=openai_api_key,
    model_name="gpt-4-turbo-preview"
)

In [6]:
# invoke통한 사전정의 prompt 기반 llm 답변 생성
print(chat.invoke(prompt).content)

Bitcoin is used more as a store of value and less as a medium of exchange or unit of account. It is primarily seen as an investment.


In [7]:
#이제 프롬프트 템플릿을 사용해서 사용자 쿼리에 따라 프롬프트가 자동으로 업데이트 되는 구조를 구현
from langchain import PromptTemplate

template = """Please answer the question based on the context below. If you can't find the information to answer the question
from the context provided, please say "I don't know".

Context: Bitcoin is the first decentralized cryptocurrency.
Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger,
called a blockchain, without central oversight.
Consensus between nodes is achieved using a computationally intensive process based on proof of work,
called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.
Based on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.
Use of bitcoin as a currency began in 2009, ith the release of its open-source implementation.
Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account.
It is mostly seen as an investment and has been described by many scholars as an economic bubble.
As bitcoin is pseudonymous, its use by criminals has attracted the attention of regulators,
leading to its ban by several countries as of 2021.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [8]:
print(
    prompt_template.format(
        query="what is bitcoin?"
    )
)

Please answer the question based on the context below. If you can't find the information to answer the question
from the context provided, please say "I don't know".

Context: Bitcoin is the first decentralized cryptocurrency.
Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger,
called a blockchain, without central oversight.
Consensus between nodes is achieved using a computationally intensive process based on proof of work,
called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.
Based on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.
Use of bitcoin as a currency began in 2009, ith the release of its open-source implementation.
Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account.
It is mostly seen as an investment and has been described by many scho

In [9]:
print(chat.invoke(prompt_template.format(
        query="What is bitcoin?"
    )))

content='Bitcoin is the first decentralized cryptocurrency, which operates on a peer-to-peer network that verifies transactions through cryptography and records them on a public distributed ledger called a blockchain, without the need for central oversight. It was invented in 2008 by an unknown person or group of people using the pseudonym Satoshi Nakamoto, and its use as a currency began in 2009 with the release of its open-source software. Bitcoin is known for using a proof of work system, called mining, to achieve consensus among nodes and secure its blockchain, a process that requires significant amounts of electricity. Although originally intended as a currency, bitcoin is currently used more as a store of value and is often regarded as an investment, despite being described by some as an economic bubble. Its pseudonymous nature has attracted regulatory attention due to its use in illicit activities, leading to its ban in several countries as of 2021.' response_metadata={'token_us

In [10]:
# 한글 case 확인
print(chat.invoke(prompt_template.format(
        query="비트코인이 뭐야?"
    )))

content='비트코인은 2008년에 사토시 나카모토라는 알려지지 않은 사람에 의해 발명된 최초의 분산형 암호화폐입니다. 비트코인 네트워크의 노드들은 중앙의 감독 없이 암호학을 통해 거래를 검증하고, 이를 블록체인이라고 하는 공개 분산 원장에 기록합니다. 노드 간의 합의는 작업 증명에 기반한 계산 집약적 과정, 즉 점점 더 많은 양의 전기를 필요로 하는 채굴을 통해 이루어지며, 이는 비트코인 블록체인의 보안을 보장합니다. 비트코인은 자유 시장 이념에 기반하여, 주로 가치 저장 수단으로 사용되며, 거래 수단이나 계산 단위로는 덜 사용됩니다. 대부분 투자로 여겨지며 많은 학자들에 의해 경제 버블로 묘사되었습니다. 비트코인은 익명성을 가지고 있어 범죄자들에 의한 사용이 규제기관들의 주목을 받았으며, 2021년 기준으로 여러 나라에서 금지되었습니다.' response_metadata={'token_usage': {'completion_tokens': 398, 'prompt_tokens': 246, 'total_tokens': 644}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5a7c792f-5828-4102-acfb-0517002cdf75-0'


In [11]:
# 프롬프트 수정 - 답변을 한국어로만 
template = """Please answer the question based on the context below. If you can't find the information to answer the question
from the context provided, please say "I don't know". You must always answer in Korean Language.

Context: Bitcoin is the first decentralized cryptocurrency.
Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger,
called a blockchain, without central oversight.
Consensus between nodes is achieved using a computationally intensive process based on proof of work,
called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.
Based on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.
Use of bitcoin as a currency began in 2009, ith the release of its open-source implementation.
Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account.
It is mostly seen as an investment and has been described by many scholars as an economic bubble.
As bitcoin is pseudonymous, its use by criminals has attracted the attention of regulators,
leading to its ban by several countries as of 2021.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

print(chat.invoke(prompt_template.format(
        query="What is bitcoin?"
    )))

content='비트코인은 첫 번째 탈중앙화 암호화폐입니다.' response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 248, 'total_tokens': 274}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-219df5e1-d552-48a4-80bf-caaa07f66ccf-0'


In [12]:
# 프롬프트 수정 - 동화책 읽는 말투
template = """Please answer the question based on the context below. If you can't find the information to answer the question
from the context provided, please say "I don't know". You must always answer in Korean language, as in fairy tale reading style.

Context: Bitcoin is the first decentralized cryptocurrency.
Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger,
called a blockchain, without central oversight.
Consensus between nodes is achieved using a computationally intensive process based on proof of work,
called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.
Based on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.
Use of bitcoin as a currency began in 2009, ith the release of its open-source implementation.
Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account.
It is mostly seen as an investment and has been described by many scholars as an economic bubble.
As bitcoin is pseudonymous, its use by criminals has attracted the attention of regulators,
leading to its ban by several countries as of 2021.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

print(chat.invoke(prompt_template.format(
        query="What is bitcoin?"
    )))

content='비트코인은 세계 최초의 탈중앙화된 암호화폐입니다. 노드들이 상호간에 네트워크를 통해 거래를 암호학을 이용하여 검증하고, 중앙의 감독 없이 공개 분산 원장인 블록체인에 기록합니다. 노드들 사이의 합의는 작업 증명에 기반한 계산 집약적인 과정, 즉 채굴을 통해 이루어지며, 이 과정은 점차 증가하는 양의 전기를 필요로 하고 비트코인 블록체인의 보안을 보장합니다. 자유 시장 이념에 기반하여 2008년에 사토시 나카모토라는 알려지지 않은 인물에 의해 발명되었으며, 2009년에 그의 오픈 소스 구현이 공개되면서 화폐로서 사용되기 시작했습니다. 현재 비트코인은 교환 수단이나 계좌 단위로서보다는 가치 저장 수단으로 더 많이 사용되며, 대부분 투자로 여겨지고, 많은 학자들에 의해 경제적 거품으로 묘사되었습니다. 비트코인이 익명성을 제공하기 때문에 범죄에 사용된 사례가 있어 규제기관의 주목을 받았고, 2021년 기준 여러 국가에서 금지되기도 했습니다.' response_metadata={'token_usage': {'completion_tokens': 462, 'prompt_tokens': 255, 'total_tokens': 717}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f5245f7d-7414-4bfc-b25b-c2f1cf8b3f93-0'


In [13]:
# Few-shot으로 톤조정하기
template = """Please answer the question based on the context below. If you can't find the information to answer the question
from the context provided, please say "I don't know". You must always answer in Korean language, as in fairy tale reading style.

Context: Bitcoin is the first decentralized cryptocurrency.
Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger,
called a blockchain, without central oversight.
Consensus between nodes is achieved using a computationally intensive process based on proof of work,
called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.
Based on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.
Use of bitcoin as a currency began in 2009, ith the release of its open-source implementation.
Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account.
It is mostly seen as an investment and has been described by many scholars as an economic bubble.
As bitcoin is pseudonymous, its use by criminals has attracted the attention of regulators,
leading to its ban by several countries as of 2021.

Question: When was bitcoin first used?
Answer: 비트코인은 2009년에 처음 사용되기 시작했답니다~

Question: Who invented Bitcoin?
Answer: 비트코인은 2008년에 사토시 나카모토라고 하는 신원미상의 사람이 발명한 코인이랍니다~

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

print(chat.invoke(prompt_template.format(
        query="What is bitcoin?"
    )))

content='비트코인은 중앙 관리 없이 암호학을 통해 거래를 검증하고, 블록체인이라고 하는 공개 분산 원장에 기록하는 최초의 탈중앙화된 암호화폐랍니다~ 노드 간의 합의는 작업 증명에 기반한 계산 집약적인 과정, 즉 채굴을 통해 이루어지며, 이는 점점 더 많은 전력을 요구하고 비트코인 블록체인의 보안을 보장한다네요. 비트코인은 자유 시장 이념을 바탕으로 만들어졌으며, 현재는 가치 저장 수단으로 더 많이 사용되며, 교환 매체나 계산 단위로는 덜 사용된다고 해요. 많은 학자들에 의해 경제적 거품으로 묘사되었고, 익명성 때문에 범죄자들이 사용하여 규제기관의 주목을 받고, 2021년까지 여러 국가에서 금지되기도 했다네요~' response_metadata={'token_usage': {'completion_tokens': 339, 'prompt_tokens': 348, 'total_tokens': 687}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-398f1bb5-0728-4bb0-a8d4-229c4da855fe-0'


In [15]:
# Langchain FewShotPromptTemplate으로 더 간편하게 만들기

from langchain import FewShotPromptTemplate
examples = [
    {
        "query" : "When was bitcoin first used?",
        "answer" : "비트코인은 2009년에 처음 사용되기 시작했답니다~"
    },{
        "query": "Who invented Bitcoin?",
        "answer": "비트코인은 2008년에 사토시 나카모토라고 하는 신원미상의 사람이 발명한 코인이랍니다~"
    }
]

example_template = """
question: {query}
answer: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """Please answer the question based on the context below. If you can't find the information to answer the question
from the context provided, please say "I don't know". You must always answer in Korean language, as in fairy tale reading style.

Context: Bitcoin is the first decentralized cryptocurrency.
Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger,
called a blockchain, without central oversight.
Consensus between nodes is achieved using a computationally intensive process based on proof of work,
called mining, that requires increasing quantities of electricity and guarantees the security of the bitcoin blockchain.
Based on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person.
Use of bitcoin as a currency began in 2009, ith the release of its open-source implementation.
Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account.
It is mostly seen as an investment and has been described by many scholars as an economic bubble.
As bitcoin is pseudonymous, its use by criminals has attracted the attention of regulators,
leading to its ban by several countries as of 2021.
"""

suffix = """
question: {query}
answer: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [16]:
query = "What is bitcoin?"

print(chat.invoke(few_shot_prompt_template.format(
        query=query)))

content='비트코인은 중앙 감독 없이 노드들이 암호학을 통해 거래를 검증하고, 그 거래 기록을 블록체인이라고 불리는 공개 분산 원장에 기록하는 세계 최초의 탈중앙화된 암호 화폐랍니다~' response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 350, 'total_tokens': 451}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-30ae07b9-0370-4a3b-ba77-6ab087edbd8e-0'
